In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Set up data


In [4]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
#icvm.mode = 'fast_pipeline_test'
icvm.mode = 'full_analysis'
#icvm.mode = 'full_pipeline_test'
icvm.group_mode = 'dichotomous'
#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV002', 'DEV280', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [6]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV004,intervention
1,DEV005,intervention
2,DEV008,intervention
3,DEV009,intervention
4,DEV010,intervention
...,...,...
238,DEV308,control
239,DEV309,intervention
240,DEV310,intervention
241,DEV311,control


In [7]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_only': condition_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols
    
    
    
    
}


# outcome_vars_to_try = [ 'bf','cancer_promoting_FFQ',
#        'NUTRIENT_DENSITY_2wkAverage']

In [8]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

EDM, BIS_11, PCS, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, NCS_total, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_FCI, age365, education_own, household_income_per_person, SST_PostErrorSlowW1_mean, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, ROC_Crave_Minus_Neutral, WTP_unhealthy_minus_healthy, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, sst_CorrectGo_striatum_joint_mask, sst_FailedStop_motor_control_striatum_joint_mask, sst_CorrectGoFollowingFailedStop_striatum_joint_mask, Planning_aggregate, Restraint_aggregate, IMI_effort_importance_aggregate, wtp_roc_koban_kober_craving_combined, birthsex_factor_Male, intervention, EDM*intervention, BIS_11*intervention, PCS*intervention, ACES_sum*intervention, BFI_agreeableness*intervention, BFI_conscientiousness*intervention, BFI_e

In [9]:
print(", ".join(dev_cv_analysis.outcome_measures.columns))

SID, bf, NUTRIENT_DENSITY_2wkAverage, ANTINUTRIENT_DENSITY_2wkAverage, total_calorie


## Body Fat Percentage

In [10]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'bf', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV002', 'DEV280', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
 attempting to predict bf with 1 predictors in the set condition_only
predictors in that set are intervention
outer split0
Only one feature, so skipping feature selection
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Time elapsed for R

doing permutation test on importance; this may take time.
Number of selected features: 1


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17dab7100>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Ridge: 85.32 seconds
Pipeline(steps=[('scaler', S

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
6,BFI_conscientiousness,None,0.016956,0.016956
50,household_income_per_person*intervention,None,0.015091,0.015091
46,RTFS_f1_minus_f2*intervention,None,0.014829,0.014829
26,sst_CorrectGo_striatum_joint_mask,None,0.014413,0.014413
41,BFI_neuroticism*intervention,None,0.013407,0.013407
17,household_income_per_person,None,0.010663,0.010663
66,birthsex_factor_Male*intervention,None,0.009985,0.009985
57,roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01*intervention,None,0.009769,0.009769
2,BIS_11,None,0.009644,0.009644
16,education_own,None,0.009421,0.009421


{'condition_only': -0.035252397998722, 'condition_inddiff': -0.060086173920266384, 'condition_inddiff_interactions': -0.04392321664586725}


## Nutrient density

In [11]:
model_outcomes = icvm.do_predictor_set_comparison(predictor_sets, 'NUTRIENT_DENSITY_2wkAverage')

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV002', 'DEV280', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 1 predictors in the set condition_only
predictors in that set are intervention
outer split0
Only one feature, so skipping feature selection
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 2

doing permutation test on importance; this may take time.
Number of selected features: 0


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17dab7100>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Ridge: 30.74 seconds
Pipeline(steps=[('scaler', S

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
37,ACES_sum*intervention,None,0.028203,0.028203
4,ACES_sum,None,0.026457,0.026457
30,Restraint_aggregate,None,0.021187,0.021187
3,PCS,None,0.006574,0.006574
36,PCS*intervention,None,0.005303,0.005303


{'condition_only': -0.017353669493918523, 'condition_inddiff': 0.004414527212305707, 'condition_inddiff_interactions': -0.02397094807636795}


## Anti-nutrient density

In [12]:
model_outcomes = icvm.do_predictor_set_comparison(predictor_sets, 'ANTINUTRIENT_DENSITY_2wkAverage')

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV002', 'DEV280', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
 attempting to predict ANTINUTRIENT_DENSITY_2wkAverage with 1 predictors in the set condition_only
predictors in that set are intervention
outer split0
Only one feature, so skipping feature selection
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalli

doing permutation test on importance; this may take time.
Number of selected features: 0


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17dab7100>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Ridge: 29.03 seconds
Pipeline(steps=[('scaler', S

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
2,BIS_11,None,0.508102,0.508102
16,education_own,None,0.495858,0.495858
49,education_own*intervention,None,0.490123,0.490123
54,ROC_Crave_Minus_Neutral*intervention,None,0.489896,0.489896
47,cancer_promoting_minus_preventing_FCI*intervention,None,0.481003,0.481003
29,Planning_aggregate,None,0.474102,0.474102
45,SRHI_healthy_minus_unhealthy*intervention,None,0.473041,0.473041
65,wtp_roc_koban_kober_craving_combined*intervention,None,0.472614,0.472614
7,BFI_extraversion,None,0.469174,0.469174
18,SST_PostErrorSlowW1_mean,None,0.461889,0.461889


{'condition_only': -0.02732281905063667, 'condition_inddiff': -0.10996406275199519, 'condition_inddiff_interactions': -0.08916982713282509}


## Calorie count

In [13]:
model_outcomes = icvm.do_predictor_set_comparison(predictor_sets, 'total_calorie')

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV002', 'DEV280', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
 attempting to predict total_calorie with 1 predictors in the set condition_only
predictors in that set are intervention
outer split0
Only one feature, so skipping feature selection
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Time el

doing permutation test on importance; this may take time.


...

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17dab7100>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Lasso())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Lasso: 19.55 seconds
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', SVR())])
{'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 5 candidates, totalling 20 fits
Pipeline(steps=[('sc

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
14,cancer_promoting_minus_preventing_FCI,None,0.004130,0.004130
66,birthsex_factor_Male*intervention,None,0.001434,0.001434
44,TESQ_E_sum*intervention,None,0.001378,0.001378
33,birthsex_factor_Male,None,0.001157,0.001157
42,BFI_openness*intervention,None,0.001011,0.001011


{'condition_only': -0.07356539440031389, 'condition_inddiff': -0.05799376009189765, 'condition_inddiff_interactions': -0.0820819279450864}
